In [1]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets




pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1800)


<h1 style="color:#FF9260;"><center>COVID-19 Interactive Analysis Dashboard</center></h1>

<h2 style="color:#B3383A;"><center>What is COVID-19?</center></h2>

<p><center style="font-size:130%;">Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS).1
'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). 3The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`</center></p>

<p style="text-align:center;"> <img src="https://www.apta.com/wp-content/uploads/home-banner-1.jpg"  /></p>


In [2]:
#width="1920" height="1080"
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
#confirmed_df.head() 

In [ ]:
#recovered_df.head()

In [ ]:
#death_df.head()

In [ ]:
#country_df.head()

In [ ]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
# country_df.head()

In [ ]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [ ]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 5px; text-align:center; '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

<h1 style="color:LightSteelBlue;"><center>COVID-19 Confirmed/Death/Recovered cases by countries</center></h1>


<h2 style="color:#F6DDCC;"> Enter number of countries you want the data for</h2>

In [ ]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
   
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

In [ ]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

<h2 style="color:#F6DDCC;">Slide to check for the worst hit countries</h2>

In [ ]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 1800)
    fig.update_layout(
        paper_bgcolor=" #FEF5E7"
    )
    fig.show();
    

def show_latest_cases(n):
    n = int(n)
    return bubble_chart(n)

interact(show_latest_cases, n='10')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

In [ ]:


def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths', 'recovered']
    colors = ['blue', 'black', 'green']
    mode_size = [6, 8,1]
    line_size = [4, 5,1]
    
    df_list = [confirmed_df, death_df , recovered_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="#D4E6F1",
        width = 1800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();


<h2 style="color:#F6DDCC;"> Check the details of your country or the World</h2> 

<p style="font-size:130%;">Enter the name of your country(in capitalized format(e.g. Italy , Tunisia , US)) and world for total cases , you can click on one or more of the variables(confirmed , death or recovered) to hide or show it's graph

In [ ]:
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

<h2 style="color:#F6DDCC;"> Top worst hit countries - Confirmed cases</h2> 

In [ ]:
def confirmed (n):
    fig =px.bar(
    sorted_country_df.head(n),
    x = "country",
    y = "confirmed",
    title= "Top worst affected countries : confirmed numbers", # the axis names
    color_discrete_sequence=["#5DADE2"], 
    height=500,
    width=1800)
    fig.update_layout(
        paper_bgcolor="#FDEDEC"
    )
    fig.show();

def show_latest_cases(n):
    n = int(n)
    return confirmed(n)

interact(show_latest_cases, n='10')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)



<h2 style="color:#F6DDCC;">Top worst hit countries - Death cases</h2> 

In [ ]:
sorted_country_df = country_df.sort_values('deaths', ascending= False)

In [ ]:
def death (n):
    fig =px.bar(
    sorted_country_df.head(n),
    x = "country",
    y = "deaths",
    title= "Top worst affected countries : death numbers", # the axis names
    color_discrete_sequence=["#E74C3C"], 
    height=500,
    width=1800)
    fig.update_layout(
        paper_bgcolor="#FDEDEC"
    )
    fig.show();

def show_latest_cases(n):
    n = int(n)
    return death(n)

interact(show_latest_cases, n='10')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

<h2 style="color:#F6DDCC;">Top  Worst hit countries - Recovering cases</h2>

In [ ]:
sorted_country_df = country_df.sort_values('recovered', ascending= False)

In [ ]:
def recover (n):
    fig = px.bar(
    sorted_country_df.head(n),
    x = "country",
    y = "recovered",
    title= "Top worst affected countries : recovered numbers", # the axis names
    color_discrete_sequence=["#1ABC9C"], 
    height=500,
    width=1800)
    fig.update_layout(
        paper_bgcolor="#E8F8F5"
    )
    fig.show();
   
    
#interact(recover , n=10)
    
def show_latest_cases(n):
    n = int(n)
    return recover(n)

interact(show_latest_cases, n='10')
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)